In [1]:
#Import packages
import pandas as pd
import numpy as np
import glob
import re
import string
import codecs

In [8]:
#Create single parliament_qs dataframe with all question data
li = []

for file in glob.glob('./india_headlines_data_pre/*.csv'):
    data = pd.read_csv(file)
    li.append(data)
    
headline = pd.concat(li, axis = 0, ignore_index = True)

In [9]:
headline

,publish_date,headline_category,headline_text
0,20010101,sports.wwe,win over cena satisfying but defeating underta...
1,20010102,bollywood,Raju Chacha
2,20010102,unknown,Status quo will not be disturbed at Ayodhya; s...
3,20010102,unknown,Fissures in Hurriyat over Pak visit
4,20010102,unknown,America's unwanted heading for India?
5,20010102,unknown,For bigwigs; it is destination Goa
6,20010102,unknown,Extra buses to clear tourist traffic
7,20010102,unknown,Dilute the power of transfers; says Riberio
8,20010102,unknown,Focus shifts to teaching of Hindi
9,20010102,unknown,IT will become compulsory in schools


In [10]:
#Code borrowed and adapted from George Chen, Carnegie Mellon University#
#Define function to remove punctuation and whitespace, and lowercase all text
def makeWordList(str_object):
    
    corpus_text = str(str_object)
    
    for c in string.punctuation:
        corpus_text = corpus_text.replace(c, "")  # -- (1)
    
    text = re.sub(r'\S*\d\S*','',corpus_text) # -- (2)
    text = re.sub(r'[^\w\s]','',text)         # -- (3)
    
    text = text.lower().split()           # -- (4)         
    
    li = []
    for token in text:
        li.append(token)

    return " ".join(li)

In [11]:
#Process the questions
processed_headline = []

for str_object in list(headline["headline_text"]):
    processed_headline.append(makeWordList(str_object))

In [13]:
#Use TfidfVectorizer to transform parliamentary questions
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=2, stop_words="english", max_df=0.8)
headline_fit = vectorizer.fit(processed_headline)
X_headline = vectorizer.fit_transform(processed_headline)

In [14]:
#!Time-consuming!#
#Create topics using LDA
num_topics = 25

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=num_topics, learning_method='online', random_state=0)
lda.fit(X_headline)

KeyboardInterrupt: 

In [ ]:
#Display top 10 words from each topic
words = list(headline_fit.vocabulary_)
topic_word_distributions = np.array([row / row.sum() for row in lda.components_])
num_top_words = 10

print('Displaying the top 10 words per topic and their probabilities within the topic...')
print()

for topic_idx in range(num_topics):
    print('[Topic ', topic_idx, ']', sep='')
    sort_indices = np.argsort(-topic_word_distributions[topic_idx])
    for rank in range(num_top_words):
        word_idx = sort_indices[rank]
        print(words[word_idx], ':', topic_word_distributions[topic_idx, word_idx])
    print()

In [23]:
lda.components_

AttributeError: 'LatentDirichletAllocation' object has no attribute 'components_'